<div style='text-align:center; padding: 20px 0;'>
<h1 style='color:#A31F34; margin-bottom:0;'>MIT AI STUDIO COURSE</h1>
<h3 style='color:#8A8B8C; margin-top:5px;'>MAS.664 / MAS.665 / EC.731 / IDS.865</h3>
<h2>RAG Workshop: Chat with the Course Website</h2>
<p style='font-size:14px; color:#666;'>Lead Professor: Ramesh Raskar, MIT Media Lab</p>
</div>

---

**Workshop Instructor:** [Brandon Sneider](https://linkedin.com/in/brandonsneider) — AI Manager at a defense technology startup, building AI systems for highly regulated environments. Brandon brings practical experience deploying LLMs and RAG pipelines where accuracy, compliance, and auditability are non-negotiable.

---

In this **30-minute hands-on workshop**, you'll build a **Retrieval-Augmented Generation (RAG)** system that crawls the [AI Studio course website](https://aiforimpact.github.io/), stores it in a vector database, and lets you **chat with the data** — all inside this notebook.

### What is RAG?

Large Language Models (LLMs) are powerful but have two critical limitations:
1. **Knowledge cutoff** — they don't know about data after their training date
2. **Hallucination** — they confidently generate plausible but incorrect information

**Retrieval-Augmented Generation (RAG)** solves both by retrieving relevant documents from a knowledge base and injecting them into the LLM's prompt as context before generating an answer.

```
User Question → [Retrieve relevant docs] → [Inject into prompt] → LLM → Grounded Answer
```

### Key Academic References

| Paper | Key Contribution |
|-------|------------------|
| Lewis et al. (2020) "[Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)" — *NeurIPS 2020* | Introduced RAG: combines retriever + generator, reducing hallucination and improving factual accuracy |
| Karpukhin et al. (2020) "[Dense Passage Retrieval for Open-Domain QA](https://arxiv.org/abs/2004.04906)" — *EMNLP 2020* | Dense embeddings outperform BM25 keyword search for passage retrieval |
| Robertson & Zaragoza (2009) "[The Probabilistic Relevance Framework: BM25 and Beyond](https://doi.org/10.1561/1500000019)" | Foundational work on BM25 scoring |
| Gao et al. (2024) "[Retrieval-Augmented Generation for Large Language Models: A Survey](https://arxiv.org/abs/2312.10997)" | Comprehensive survey: Naive RAG, Advanced RAG, Modular RAG |
| Muennighoff et al. (2023) "[MTEB: Massive Text Embedding Benchmark](https://arxiv.org/abs/2210.07316)" | Benchmark for comparing embedding models across 56 datasets |

### Learning Outcomes

By the end of this workshop, you will be able to:

1. **Explain** what RAG is and why it matters for business (reducing hallucination, grounding LLMs in private data)
2. **Compare** embedding models and understand when to fine-tune vs use off-the-shelf
3. **Build** a document ingestion pipeline that crawls a website, chunks content, and generates embeddings
4. **Understand** the difference between BM25 keyword search and semantic vector search, and why hybrid search outperforms either alone
5. **Implement** a working RAG chatbot backed by SQLite + sqlite-vec
6. **Evaluate** RAG quality and use AI to tune hyperparameters

### Tech Stack

| Component | Technology | Why |
|-----------|-----------|-----|
| Database | **SQLite + sqlite-vec** | Zero infrastructure, runs anywhere, scales to millions of vectors |
| Keyword Search | **FTS5 (BM25)** | Built into SQLite, great for exact term matching |
| Vector Search | **sqlite-vec** | Native cosine similarity, compact binary storage |
| Embeddings | **MiniLM-L6-v2** (local) | No API key needed, runs on CPU, 384 dimensions |
| LLM | **OpenRouter** | Access to frontier models, OpenAI-compatible API |
| Web Scraping | **BeautifulSoup** | Parse the AI Studio website HTML |
| Chat UI | **IPython widgets** | Interactive chat right in the notebook |

---
## Part 1: Setup & Installation (2 min)

We install everything we need in one cell. This takes ~30 seconds on Colab.

In [ ]:
%%capture
# Install all dependencies (run once)
!pip install -q sqlite-vec fastembed numpy openai requests beautifulsoup4 lxml

In [ ]:
import json
import os
import re
import sqlite3
import struct
from datetime import datetime
from pathlib import Path
from urllib.parse import urljoin, urlparse

import numpy as np
import requests
import sqlite_vec
from bs4 import BeautifulSoup
from fastembed import TextEmbedding

print("All libraries loaded!")

---
## Part 2: Initialize the Vector Database (3 min)

We use **SQLite** with two extensions:
- **FTS5** — Full-Text Search for BM25 keyword matching (built into SQLite)
- **sqlite-vec** — Vector similarity search using cosine distance

This gives us a **hybrid search** system in a single file — no external database servers needed.

> **Why hybrid?** Karpukhin et al. (2020) showed dense retrieval beats BM25 for semantic queries,
> but BM25 still wins for exact name/term lookups. Combining both gets the best of each.

| Query Example | BM25 (keyword) | Semantic (vector) | Best Approach |
|--------------|:-:|:-:|---|
| "Ramesh Raskar" | Exact match | May miss | **BM25** |
| "Who teaches AI ethics?" | No exact terms | Understands meaning | **Semantic** |
| "MIT Media Lab AI course" | Partial match | Related concepts | **Hybrid** |

In [ ]:
DATABASE_PATH = Path("ai_studio_rag.db")

def init_database(db_path: Path) -> sqlite3.Connection:
    """Create database with embeddings table, FTS5 for BM25, and vec0 for vectors."""
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row

    # Load sqlite-vec extension
    conn.enable_load_extension(True)
    sqlite_vec.load(conn)
    conn.enable_load_extension(False)

    cursor = conn.cursor()

    # Metadata table — stores the actual content and source info
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            url TEXT NOT NULL,
            page_title TEXT,
            section_title TEXT,
            content TEXT NOT NULL,
            content_type TEXT DEFAULT 'text',
            metadata TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)

    # Vector table — 384 dimensions for MiniLM-L6-v2 embeddings
    cursor.execute("""
        CREATE VIRTUAL TABLE IF NOT EXISTS vec_documents USING vec0(
            embedding float[384] distance_metric=cosine
        )
    """)

    # FTS5 table — for BM25 keyword search
    cursor.execute("""
        CREATE VIRTUAL TABLE IF NOT EXISTS documents_fts USING fts5(
            content,
            page_title,
            section_title,
            content='documents',
            content_rowid='id'
        )
    """)

    # Triggers to keep FTS5 in sync automatically
    cursor.execute("""
        CREATE TRIGGER IF NOT EXISTS docs_ai AFTER INSERT ON documents BEGIN
            INSERT INTO documents_fts(rowid, content, page_title, section_title)
            VALUES (new.id, new.content, new.page_title, new.section_title);
        END
    """)
    cursor.execute("""
        CREATE TRIGGER IF NOT EXISTS docs_ad AFTER DELETE ON documents BEGIN
            INSERT INTO documents_fts(documents_fts, rowid, content, page_title, section_title)
            VALUES ('delete', old.id, old.content, old.page_title, old.section_title);
        END
    """)

    conn.commit()
    return conn

# Initialize!
db = init_database(DATABASE_PATH)
print(f"Database created at: {DATABASE_PATH}")
print(f"sqlite-vec loaded — ready for vector search!")

---
## Part 3: Crawl the AI Studio Website (5 min)

We'll scrape every page of [aiforimpact.github.io](https://aiforimpact.github.io/) including:
- Course overview and pillars
- Speaker & mentor bios (40+ people)
- Schedule and registration info
- Past semester archives (Spring 2024 → Spring 2026)
- Embedded YouTube video references
- Image descriptions

### The RAG Ingestion Pipeline

```
Website → Crawl HTML → Extract & Clean → Chunk by Section → Embed → Store in SQLite
```

**Chunking strategy**: We split by HTML sections (headers), keeping each chunk semantically coherent.
This matters — Gao et al. (2024) show that chunk size and boundary selection significantly impact retrieval quality.

In [ ]:
# ── Website Crawler ──────────────────────────────────────────────────────────

SITE_PAGES = [
    "https://aiforimpact.github.io/",
    "https://aiforimpact.github.io/spring26.html",
    "https://aiforimpact.github.io/fall25.html",
    "https://aiforimpact.github.io/spring25.html",
    "https://aiforimpact.github.io/fall24.html",
    "https://aiforimpact.github.io/spring24.html",
    "https://aiforimpact.github.io/fall23.html",
]

def crawl_page(url: str) -> dict:
    """Fetch and parse a single page, extracting structured content."""
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "lxml")

    title = soup.title.string.strip() if soup.title else url.split('/')[-1]

    # Remove script and style elements
    for tag in soup(["script", "style", "nav"]):
        tag.decompose()

    chunks = []

    # ── Extract speaker/mentor bios ──
    for card in soup.select(".card, .speaker-card, .col-md-3, .col-lg-3"):
        img = card.find("img")
        text_parts = [t.strip() for t in card.stripped_strings]
        if text_parts and len(text_parts) >= 2:
            name = text_parts[0]
            role = " ".join(text_parts[1:])
            img_url = urljoin(url, img["src"]) if img and img.get("src") else None
            bio_text = f"Speaker/Mentor: {name}. Role: {role}."
            if img_url:
                bio_text += f" Photo: {img_url}"
            chunks.append({
                "content": bio_text,
                "section_title": f"Bio: {name}",
                "content_type": "bio",
                "metadata": {"name": name, "role": role, "image_url": img_url}
            })

    # ── Extract YouTube videos ──
    for iframe in soup.find_all("iframe"):
        src = iframe.get("src", "")
        if "youtube" in src or "youtu.be" in src:
            video_title = iframe.get("title", "Course Video")
            chunks.append({
                "content": f"Video: {video_title}. URL: {src}. This is an embedded video from the AI Studio course page.",
                "section_title": f"Video: {video_title}",
                "content_type": "video",
                "metadata": {"video_url": src, "title": video_title}
            })

    # ── Extract images with context ──
    for img in soup.find_all("img"):
        src = img.get("src", "")
        alt = img.get("alt", "")
        if src and not src.startswith("data:") and alt:
            img_url = urljoin(url, src)
            parent_text = img.parent.get_text(strip=True)[:200] if img.parent else ""
            chunks.append({
                "content": f"Image: {alt}. URL: {img_url}. Context: {parent_text}",
                "section_title": f"Image: {alt[:50]}",
                "content_type": "image",
                "metadata": {"image_url": img_url, "alt_text": alt}
            })

    # ── Extract main text content by sections ──
    main = soup.find("main") or soup.find("body")
    if main:
        current_section = "Overview"
        current_text = []

        for element in main.find_all(["h1", "h2", "h3", "h4", "p", "li", "td", "th"]):
            if element.name in ["h1", "h2", "h3"]:
                if current_text:
                    text = "\n".join(current_text).strip()
                    if len(text) > 30:
                        chunks.append({
                            "content": f"{current_section}\n\n{text}",
                            "section_title": current_section,
                            "content_type": "text",
                            "metadata": {}
                        })
                current_section = element.get_text(strip=True)
                current_text = []
            else:
                text = element.get_text(strip=True)
                if text and len(text) > 5:
                    current_text.append(text)

        if current_text:
            text = "\n".join(current_text).strip()
            if len(text) > 30:
                chunks.append({
                    "content": f"{current_section}\n\n{text}",
                    "section_title": current_section,
                    "content_type": "text",
                    "metadata": {}
                })

    return {"url": url, "title": title, "chunks": chunks}


def crawl_site(pages: list[str]) -> list[dict]:
    """Crawl all pages and return flat list of chunks."""
    all_chunks = []
    for url in pages:
        print(f"  Crawling: {url}")
        try:
            page = crawl_page(url)
            for chunk in page["chunks"]:
                chunk["url"] = url
                chunk["page_title"] = page["title"]
            all_chunks.extend(page["chunks"])
            print(f"    -> {len(page['chunks'])} chunks extracted")
        except Exception as e:
            print(f"    Error: {e}")
    return all_chunks


print("Crawling the AI Studio website...\n")
all_chunks = crawl_site(SITE_PAGES)

# Summary
types = {}
for c in all_chunks:
    types[c["content_type"]] = types.get(c["content_type"], 0) + 1

print(f"\n{'='*50}")
print(f"Total chunks extracted: {len(all_chunks)}")
for t, count in sorted(types.items()):
    print(f"  {t}: {count}")

---
## Part 4: Embedding Models Deep Dive & Store (5 min)

Now we convert each text chunk into a **384-dimensional vector** using the MiniLM-L6-v2 model.

**How embeddings work:**
- Text → Neural network → Dense vector (array of 384 numbers)
- Semantically similar texts produce vectors that are **close together** in vector space
- "Who teaches the course?" and "Professor Ramesh Raskar leads the class" will have similar embeddings, even though they share few words

### Why MiniLM-L6-v2? Embedding Model Comparison

Choosing an embedding model is one of the most impactful decisions in a RAG system. Here's how the major options compare:

| Model | Dims | Size | Cost | Speed | Multilingual | Best For |
|-------|------|------|------|-------|-------------|----------|
| **all-MiniLM-L6-v2** (ours) | 384 | 80 MB | Free (local) | ~50ms | English only | Prototyping, workshops, CPU |
| **BGE-M3** (BAAI) | 1024 | 2.2 GB | Free (local) | ~200ms | 100+ languages | Production multilingual RAG |
| **Amazon Titan Embeddings** | 1024 | API | $0.0001/1K tok | ~100ms | 25+ languages | AWS-integrated pipelines |
| **OpenAI text-embedding-3-large** | 3072 | API | $0.00013/1K tok | ~80ms | Good | Highest quality, cost-tolerant |
| **Cohere embed-v3** | 1024 | API | $0.0001/1K tok | ~90ms | 100+ languages | Search-optimized |

**We chose MiniLM-L6-v2 because:**
1. **Zero cost** — no API key needed for embeddings, runs entirely on CPU
2. **Fast** — smallest model, perfect for a 30-min workshop
3. **Good enough** — for English-only <100K docs, quality is comparable to larger models
4. **384 dimensions** — smaller vectors = faster search, less storage (vs 1024 or 3072)

> **When to upgrade:** For multilingual, use BGE-M3. For max quality, use OpenAI's `text-embedding-3-large`.
> See Muennighoff et al. (2023) "[MTEB: Massive Text Embedding Benchmark](https://arxiv.org/abs/2210.07316)" for comprehensive benchmarks.

### When and How to Fine-Tune an Embedding Model

**When fine-tuning helps** (Wang et al. 2024, "[Improving Text Embeddings with LLMs](https://arxiv.org/abs/2401.00368)"):
- Your domain has **specialized vocabulary** (legal, medical, finance) that general models don't capture
- You have **query-document pairs** showing what users search for vs what they should find
- General models score below ~0.7 accuracy on your test queries
- Example: A legal RAG where "consideration" means contract terms, not "thinking about"

**When fine-tuning is NOT worth it:**
- Your content is general English (like our course website)
- You have fewer than ~1,000 labeled query-document pairs
- Switching to a larger pre-trained model would solve the problem
- You're still iterating on chunking strategy or retrieval logic

**How fine-tuning works (conceptual):**
```
1. Collect pairs:    (query, relevant_document, irrelevant_document)
2. Contrastive loss: Push relevant pairs closer, irrelevant pairs apart
3. Result:           Model now "understands" your domain's similarity
```

Tools: [Sentence-Transformers fine-tuning](https://www.sbert.net/docs/training/overview.html), or generate synthetic pairs with LLMs (Wang et al. 2024).

In [ ]:
# Load the embedding model (downloads ~80MB on first run)
print("Loading MiniLM-L6-v2 embedding model (384 dimensions, ONNX runtime)...")
embedding_model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Model loaded!\n")


def generate_embedding(text: str) -> list[float]:
    """Generate a 384-dim embedding for a single text."""
    return list(embedding_model.embed([text]))[0].tolist()


def generate_embeddings_batch(texts: list[str]) -> list[list[float]]:
    """Batch embed for efficiency."""
    if not texts:
        return []
    return [emb.tolist() for emb in embedding_model.embed(texts)]


def serialize_embedding(embedding: list[float]) -> bytes:
    """Pack embedding as binary for sqlite-vec."""
    return struct.pack(f'{len(embedding)}f', *embedding)


# ── Quick demo: see how embeddings capture meaning ──
demo_texts = [
    "Professor Raskar teaches AI at MIT",
    "The lead instructor of the course is from Media Lab",
    "I had pizza for lunch today",
]
demo_embs = generate_embeddings_batch(demo_texts)

print("Embedding similarity demo:")
print(f"  Text A: '{demo_texts[0]}'")
print(f"  Text B: '{demo_texts[1]}'  (related)")
print(f"  Text C: '{demo_texts[2]}'  (unrelated)")

from numpy import dot
from numpy.linalg import norm
def cosine_sim(a, b): return dot(a, b) / (norm(a) * norm(b))

print(f"\n  Similarity A<->B (related):   {cosine_sim(demo_embs[0], demo_embs[1]):.3f}")
print(f"  Similarity A<->C (unrelated): {cosine_sim(demo_embs[0], demo_embs[2]):.3f}")
print(f"  (Higher = more similar. Related texts cluster together!)")

In [ ]:
# ── Ingest all chunks into the database ──
print("Embedding and storing all chunks...")

BATCH_SIZE = 32
total_stored = 0

for i in range(0, len(all_chunks), BATCH_SIZE):
    batch = all_chunks[i:i + BATCH_SIZE]
    texts = [c["content"] for c in batch]
    embeddings = generate_embeddings_batch(texts)

    cursor = db.cursor()
    for chunk, emb in zip(batch, embeddings):
        cursor.execute("""
            INSERT INTO documents (url, page_title, section_title, content, content_type, metadata)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            chunk["url"],
            chunk["page_title"],
            chunk["section_title"],
            chunk["content"],
            chunk["content_type"],
            json.dumps(chunk["metadata"]),
        ))
        rowid = cursor.lastrowid
        cursor.execute("""
            INSERT INTO vec_documents (rowid, embedding) VALUES (?, ?)
        """, (rowid, serialize_embedding(emb)))

    db.commit()
    total_stored += len(batch)
    print(f"  Stored {total_stored}/{len(all_chunks)} chunks...")

print(f"\nDone! {total_stored} chunks embedded and stored in {DATABASE_PATH}")
print(f"Database size: {DATABASE_PATH.stat().st_size / 1024:.0f} KB")

---
## Part 5: Hybrid Search — BM25 + Semantic (5 min)

This is the **retrieval** core of RAG. We implement two search strategies and combine them:

### BM25 (Keyword Search)
- Uses SQLite's **FTS5** extension
- Scores based on term frequency / inverse document frequency
- Great for exact names, acronyms, specific terms
- Based on Robertson & Zaragoza (2009)

### Semantic Search (Vector Similarity)
- Uses **sqlite-vec**'s native cosine distance
- Finds conceptually similar content even with different wording
- Based on Karpukhin et al. (2020) Dense Passage Retrieval

### Hybrid Fusion
We normalize both scores to [0, 1] and combine with configurable weights:

```
final_score = keyword_weight * BM25_score + semantic_weight * cosine_similarity
```

In [ ]:
# ── Search Functions ──────────────────────────────────────────────────────────

def bm25_search(conn, query: str, limit: int = 50) -> dict[int, float]:
    """BM25 keyword search via FTS5. Returns {doc_id: score}."""
    cursor = conn.cursor()
    safe_query = query.replace('"', '""')
    try:
        cursor.execute("""
            SELECT rowid, bm25(documents_fts) as score
            FROM documents_fts
            WHERE documents_fts MATCH ?
            LIMIT ?
        """, (safe_query, limit))
        return {row[0]: row[1] for row in cursor.fetchall()}
    except sqlite3.OperationalError:
        return {}


def semantic_search(conn, query_embedding: list[float], limit: int = 50) -> dict[int, float]:
    """Cosine similarity search via sqlite-vec. Returns {doc_id: distance}."""
    cursor = conn.cursor()
    cursor.execute("""
        SELECT rowid, distance
        FROM vec_documents
        WHERE embedding MATCH ? AND k = ?
        ORDER BY distance
    """, (serialize_embedding(query_embedding), limit))
    return {row[0]: row[1] for row in cursor.fetchall()}


def normalize_scores(scores: dict[int, float], higher_is_better: bool = True) -> dict[int, float]:
    """Normalize scores to [0, 1]."""
    if not scores:
        return {}
    vals = list(scores.values())
    min_v, max_v = min(vals), max(vals)
    if min_v == max_v:
        return {k: 1.0 for k in scores}
    if higher_is_better:
        return {k: (v - min_v) / (max_v - min_v) for k, v in scores.items()}
    else:
        return {k: (max_v - v) / (max_v - min_v) for k, v in scores.items()}


def hybrid_search(
    conn,
    query: str,
    query_embedding: list[float],
    keyword_weight: float = 0.3,
    semantic_weight: float = 0.7,
    top_k: int = 5,
) -> list[dict]:
    """Combine BM25 + semantic search with weighted fusion."""
    bm25_raw = bm25_search(conn, query)
    bm25_norm = normalize_scores(bm25_raw, higher_is_better=False)

    sem_raw = semantic_search(conn, query_embedding)
    sem_norm = normalize_scores(sem_raw, higher_is_better=False)

    all_ids = set(bm25_norm.keys()) | set(sem_norm.keys())
    if not all_ids:
        return []

    cursor = conn.cursor()
    placeholders = ",".join("?" * len(all_ids))
    cursor.execute(f"""
        SELECT id, url, page_title, section_title, content, content_type, metadata
        FROM documents WHERE id IN ({placeholders})
    """, list(all_ids))
    cols = ["id","url","page_title","section_title","content","content_type","metadata"]
    docs = {row[0]: dict(zip(cols, row)) for row in cursor.fetchall()}

    results = []
    for doc_id in all_ids:
        bm25_s = bm25_norm.get(doc_id, 0.0)
        sem_s = sem_norm.get(doc_id, 0.0)
        final = keyword_weight * bm25_s + semantic_weight * sem_s
        doc = docs.get(doc_id, {})
        results.append({**doc, "bm25_score": bm25_s, "semantic_score": sem_s, "final_score": final})

    results.sort(key=lambda x: x["final_score"], reverse=True)
    return results[:top_k]


# ── Test it! ──
test_queries = [
    "Who is the lead professor of the course?",
    "What are the course pillars?",
    "Tell me about the guest speakers from venture capital",
]

for q in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {q}")
    print(f"{'='*60}")
    emb = generate_embedding(q)
    results = hybrid_search(db, q, emb, top_k=3)
    for i, r in enumerate(results, 1):
        print(f"  {i}. [{r.get('content_type','?')}] Score: {r['final_score']:.3f} "
              f"(BM25: {r['bm25_score']:.2f}, Semantic: {r['semantic_score']:.2f})")
        content_preview = r.get('content', '')[:120].replace('\n', ' ')
        print(f"     {content_preview}...")

---
## Part 6: Connect the LLM for Generation (3 min)

Now we add the **G** in RA**G** — we take retrieved context and use an LLM to generate a grounded answer.

We use **OpenRouter** which provides access to frontier models via an OpenAI-compatible API.

### The RAG Prompt Pattern

```
System: You are a helpful assistant for the MIT AI Studio course.
        Answer ONLY using the provided context.

User:   Context: [retrieved documents]
        Question: [user's question]
```

This is the key insight from Lewis et al. (2020): by conditioning generation on retrieved evidence, the model produces **factual, verifiable answers** rather than hallucinating.

In [ ]:
from google.colab import userdata
from openai import OpenAI

# ── Get API key from Colab Secrets ──
# Go to the key icon in the left sidebar -> add OPENROUTER_API_KEY
# Get a free key at https://openrouter.ai/keys
try:
    api_key = userdata.get("OPENROUTER_API_KEY")
except Exception:
    api_key = None

if not api_key:
    api_key = input("Enter your OpenRouter API key (get one at https://openrouter.ai/keys): ").strip()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)
print("LLM client connected!")

In [ ]:
# ── RAG Answer Function ───────────────────────────────────────────────────────

SYSTEM_PROMPT = """You are a helpful AI assistant for the MIT AI Studio course
(MAS.664 / MAS.665 / EC.731 / IDS.865), taught at MIT Media Lab by Professor Ramesh Raskar.

Rules:
- Answer ONLY using the provided context. Do not use outside knowledge.
- If the context mentions images or videos, include the URLs so the user can view them.
- If the context doesn't contain enough information, say so clearly.
- Be concise but thorough. Use bullet points for lists.
- When mentioning people, include their role/affiliation if available."""


def format_context(results: list[dict], max_chars: int = 4000) -> str:
    """Format retrieved documents into context for the LLM."""
    if not results:
        return "No relevant documents found."
    parts = []
    chars = 0
    for i, r in enumerate(results, 1):
        entry = f"[Source {i}: {r.get('page_title', '?')} > {r.get('section_title', '?')}]\n{r.get('content', '')}"
        if chars + len(entry) > max_chars:
            break
        parts.append(entry)
        chars += len(entry)
    return "\n\n".join(parts)


def rag_answer(question: str, conn=db, top_k: int = 5) -> tuple[str, list[dict]]:
    """Full RAG pipeline: retrieve -> format -> generate."""
    query_emb = generate_embedding(question)
    results = hybrid_search(conn, question, query_emb, top_k=top_k)
    context = format_context(results)

    response = client.chat.completions.create(
        model="google/gemini-2.0-flash-001",
        max_tokens=800,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"},
        ],
    )
    answer = response.choices[0].message.content.strip()
    return answer, results


# ── Quick test ──
question = "Who are the lead instructors and what is the course about?"
answer, sources = rag_answer(question)
print(f"Q: {question}\n")
print(f"A: {answer}\n")
print(f"Sources used: {len(sources)} documents")
for s in sources:
    print(f"  - [{s.get('content_type','')}] {s.get('section_title', '')[:60]} (score: {s['final_score']:.2f})")

---
## Part 7: Interactive Chat Interface (5 min)

Now let's build an interactive chat — right here in the notebook!

Try asking:
- "Who are the venture capital speakers?"
- "What's the difference between Spring 2025 and Fall 2025?"
- "What are the course pillars?"
- "Tell me about the demo day"
- "Are there any videos I can watch?"

In [ ]:
# ── In-Notebook Chat UI ───────────────────────────────────────────────────────

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

chat_output = widgets.Output(layout=widgets.Layout(
    width='100%', min_height='300px', max_height='500px',
    overflow_y='auto', border='1px solid #444', padding='10px',
))
input_box = widgets.Text(
    placeholder='Ask anything about the MIT AI Studio course...',
    layout=widgets.Layout(width='85%'),
)
send_btn = widgets.Button(description='Send', button_style='primary',
                          layout=widgets.Layout(width='14%'))
show_sources = widgets.Checkbox(value=False, description='Show sources', indent=False)

chat_history = []

def render_chat():
    with chat_output:
        clear_output()
        for msg in chat_history:
            if msg["role"] == "user":
                display(HTML(f"<div style='margin:8px 0;padding:10px 14px;background:#1a3a5c;"
                    f"border-radius:12px;color:white;max-width:80%;margin-left:auto;"
                    f"text-align:right;font-size:14px;'><b>You:</b> {msg['content']}</div>"))
            else:
                sources_html = ""
                if show_sources.value and msg.get("sources"):
                    src_items = "".join(
                        f"<br>- [{s.get('content_type','')}] {s.get('section_title','')[:50]} (score: {s['final_score']:.2f})"
                        for s in msg["sources"]
                    )
                    sources_html = f"<br><details><summary><small>Sources</small></summary><small>{src_items}</small></details>"
                display(HTML(f"<div style='margin:8px 0;padding:10px 14px;background:#2d2d2d;"
                    f"border-radius:12px;color:#e0e0e0;max-width:90%;font-size:14px;"
                    f"line-height:1.5;'><b>AI Studio Assistant:</b><br>{msg['content']}{sources_html}</div>"))

def on_send(btn):
    question = input_box.value.strip()
    if not question:
        return
    input_box.value = ""
    chat_history.append({"role": "user", "content": question})
    render_chat()
    with chat_output:
        display(HTML("<div style='color:#888;padding:5px;font-style:italic;'>Searching & generating...</div>"))
    try:
        answer, sources = rag_answer(question)
        chat_history.append({"role": "assistant", "content": answer, "sources": sources})
    except Exception as e:
        chat_history.append({"role": "assistant", "content": f"Error: {e}", "sources": []})
    render_chat()

send_btn.on_click(on_send)
input_box.on_submit(lambda _: on_send(None))
show_sources.observe(lambda _: render_chat(), names='value')

header = widgets.HTML("<h3 style='margin:0;padding:8px 0;'>Chat with the AI Studio Course</h3>")
display(widgets.VBox([header, chat_output, widgets.HBox([input_box, send_btn]), widgets.HBox([show_sources])]))

chat_history.append({
    "role": "assistant",
    "content": "Welcome! I've ingested the entire <b>MIT AI Studio course website</b> "
               "(7 pages, all semesters, 40+ speaker bios, videos, and images). "
               "Ask me anything!<br><br>"
               "<b>Try:</b> 'Who are the VC speakers?' or 'What is the course about?' "
               "or 'How do I register for Spring 2026?'",
    "sources": []
})
render_chat()

---
## Part 8: Understanding What Just Happened — RAG Recap

Let's trace what happens when you type a question in the chat:

```
 1. EMBED        Your question -> 384-dim vector
                 "Who teaches AI ethics?" -> [0.03, -0.12, ...]

 2. RETRIEVE     Hybrid search over the database
                 BM25:     keyword match on "teaches", "AI"
                 Semantic: cosine similarity to find related
                 -> Top 5 most relevant chunks

 3. AUGMENT      Inject retrieved chunks into the LLM prompt
                 "Context: [Source 1: ...] [Source 2: ...]"

 4. GENERATE     LLM produces answer grounded in evidence
                 -> Factual, verifiable, no hallucination
```

### Why RAG Matters for Business (MBA Perspective)

| Use Case | Without RAG | With RAG |
|----------|-------------|----------|
| Customer support chatbot | Generic answers, can't access your docs | Answers from your actual knowledge base |
| Legal document review | May hallucinate case law | Grounded in your actual contracts |
| Internal company wiki Q&A | Outdated training data | Always current with your docs |
| Due diligence research | May confuse companies | Grounded in actual filings |

### Key Takeaways

1. **RAG = Retrieve + Augment + Generate** — give the LLM the right context, get grounded answers
2. **Hybrid search > either alone** — combine keyword (BM25) and semantic (vector) for best results
3. **Embeddings are the bridge** — they convert text to numbers so we can measure similarity
4. **Model choice matters** — MiniLM for prototypes, BGE-M3/OpenAI for production
5. **The entire system runs locally** — embeddings are local, only the LLM call needs an API

---
## Exercises

### Exercise 1: Compare Search Strategies
Run the cell below to see how BM25-only, semantic-only, and hybrid search differ.

In [ ]:
# Exercise 1: Compare search strategies for different query types

test_cases = [
    ("Ramesh Raskar", "Exact name lookup - BM25 should excel"),
    ("venture capital investors in AI", "Conceptual query - semantic should excel"),
    ("MIT Media Lab course spring 2026", "Mix of specific + conceptual - hybrid wins"),
]

for query, description in test_cases:
    print(f"\n{'='*70}")
    print(f"Query: \"{query}\"")
    print(f"Expected: {description}")
    print(f"{'='*70}")

    emb = generate_embedding(query)

    r_bm25 = hybrid_search(db, query, emb, keyword_weight=1.0, semantic_weight=0.0, top_k=3)
    r_sem = hybrid_search(db, query, emb, keyword_weight=0.0, semantic_weight=1.0, top_k=3)
    r_hyb = hybrid_search(db, query, emb, keyword_weight=0.3, semantic_weight=0.7, top_k=3)

    for label, results in [("BM25 Only", r_bm25), ("Semantic Only", r_sem), ("Hybrid", r_hyb)]:
        print(f"\n  {label}:")
        if not results:
            print("    (no results)")
        for r in results[:2]:
            print(f"    - {r.get('section_title', '?')[:50]} (score: {r['final_score']:.3f})")

### Exercise 2: Add Your Own Content
Add a new document to the knowledge base and query it.

In [ ]:
# Exercise 2: Add your own content to the RAG system

my_doc = """
Replace this with your own text! For example, paste your startup idea,
a paragraph from a paper you're reading, or notes from a lecture.
"""

emb = generate_embedding(my_doc)
cursor = db.cursor()
cursor.execute("""
    INSERT INTO documents (url, page_title, section_title, content, content_type, metadata)
    VALUES (?, ?, ?, ?, ?, ?)
""", ("user://custom", "My Document", "Custom Content", my_doc, "text", "{}"))
rowid = cursor.lastrowid
cursor.execute("INSERT INTO vec_documents (rowid, embedding) VALUES (?, ?)",
               (rowid, serialize_embedding(emb)))
db.commit()
print(f"Added your document (id={rowid}). Now try asking about it in the chat above!")

### Exercise 3: Let AI Tune Your RAG Hyperparameters

Our RAG system has several "knobs" (hyperparameters) that affect quality. Right now they're set to reasonable defaults — but are they optimal for *this* dataset?

**The hyperparameters we can tune:**
- `keyword_weight` / `semantic_weight` — balance between BM25 and vector search
- `top_k` — how many documents to retrieve

Below, we first test with **untuned defaults**, then ask the LLM to analyze results and suggest better values. This is a real technique used in production RAG systems!

In [ ]:
# ── Exercise 3: Untuned vs AI-Tuned Hyperparameters ─────────────────────────

EVAL_QUERIES = [
    {"query": "Who is the lead professor?",
     "expected_keywords": ["ramesh", "raskar", "media lab"]},
    {"query": "What are the three course pillars?",
     "expected_keywords": ["innovation", "human centered", "technical"]},
    {"query": "Which venture capitalists are speakers?",
     "expected_keywords": ["khosla", "lux", "pillar"]},
    {"query": "How do I register for Spring 2026?",
     "expected_keywords": ["questionnaire", "register", "step"]},
    {"query": "What happens at demo day?",
     "expected_keywords": ["demo", "presentations", "investors"]},
]

def score_results(results: list[dict], expected_keywords: list[str]) -> float:
    """Score retrieval: what fraction of expected keywords appear in top results?"""
    if not results:
        return 0.0
    all_content = " ".join(r.get("content", "") for r in results).lower()
    hits = sum(1 for kw in expected_keywords if kw.lower() in all_content)
    return hits / len(expected_keywords)

def evaluate_config(kw_w, sem_w, top_k):
    """Run all eval queries and return average retrieval score."""
    scores = []
    for eq in EVAL_QUERIES:
        emb = generate_embedding(eq["query"])
        results = hybrid_search(db, eq["query"], emb,
                               keyword_weight=kw_w, semantic_weight=sem_w, top_k=top_k)
        scores.append(score_results(results, eq["expected_keywords"]))
    return sum(scores) / len(scores)

# ── Step 1: Test with UNTUNED defaults ──
print("=" * 60)
print("STEP 1: Testing with UNTUNED defaults")
print("  keyword_weight=0.3, semantic_weight=0.7, top_k=5")
print("=" * 60)

untuned_score = evaluate_config(0.3, 0.7, 5)
print(f"\n  Overall retrieval score: {untuned_score:.1%}\n")

for eq in EVAL_QUERIES:
    emb = generate_embedding(eq["query"])
    results = hybrid_search(db, eq["query"], emb, keyword_weight=0.3, semantic_weight=0.7, top_k=5)
    s = score_results(results, eq["expected_keywords"])
    status = "PASS" if s >= 0.66 else "WEAK" if s > 0 else "FAIL"
    print(f"  [{status}] \"{eq['query']}\" -> {s:.0%} keywords found")

# ── Step 2: Ask AI to suggest better hyperparameters ──
print(f"\n{'=' * 60}")
print("STEP 2: Asking AI to analyze and suggest better config...")
print("=" * 60)

tuning_prompt = f"""You are optimizing a RAG system's hyperparameters.
The system uses hybrid search combining BM25 keyword search and semantic vector search.

Current config: keyword_weight=0.3, semantic_weight=0.7, top_k=5
Current retrieval score: {untuned_score:.1%}

Eval results:
"""
for eq in EVAL_QUERIES:
    emb = generate_embedding(eq["query"])
    results = hybrid_search(db, eq["query"], emb, keyword_weight=0.3, semantic_weight=0.7, top_k=5)
    s = score_results(results, eq["expected_keywords"])
    top_sections = [r.get("section_title", "?")[:40] for r in results[:3]]
    tuning_prompt += (f"\n- Query: \"{eq['query']}\"\n"
                      f"  Expected keywords: {eq['expected_keywords']}\n"
                      f"  Score: {s:.0%}, Top results: {top_sections}\n")

tuning_prompt += """\nSuggest improved hyperparameters. Consider:
- If name lookups fail, increase keyword_weight
- If conceptual queries fail, increase semantic_weight
- If not enough context, increase top_k (max 10)

Respond ONLY with JSON: {"keyword_weight": 0.4, "semantic_weight": 0.6, "top_k": 7, "reasoning": "brief"}"""

response = client.chat.completions.create(
    model="google/gemini-2.0-flash-001",
    max_tokens=200,
    messages=[{"role": "user", "content": tuning_prompt}],
)
ai_response = response.choices[0].message.content.strip()
print(f"\n  AI suggestion: {ai_response}\n")

# Parse and test
json_match = re.search(r'\{[^}]+\}', ai_response)
if json_match:
    suggestion = json.loads(json_match.group())
    new_kw = suggestion.get("keyword_weight", 0.4)
    new_sem = suggestion.get("semantic_weight", 0.6)
    new_topk = min(suggestion.get("top_k", 7), 10)

    print(f"{'=' * 60}")
    print(f"STEP 3: Testing with AI-TUNED config")
    print(f"  keyword_weight={new_kw}, semantic_weight={new_sem}, top_k={new_topk}")
    print(f"{'=' * 60}")

    tuned_score = evaluate_config(new_kw, new_sem, new_topk)
    print(f"\n  Overall retrieval score: {tuned_score:.1%}\n")

    for eq in EVAL_QUERIES:
        emb = generate_embedding(eq["query"])
        results = hybrid_search(db, eq["query"], emb,
                               keyword_weight=new_kw, semantic_weight=new_sem, top_k=new_topk)
        s = score_results(results, eq["expected_keywords"])
        status = "PASS" if s >= 0.66 else "WEAK" if s > 0 else "FAIL"
        print(f"  [{status}] \"{eq['query']}\" -> {s:.0%} keywords found")

    # Summary
    print(f"\n{'=' * 60}")
    print(f"SUMMARY")
    print(f"  Untuned score:  {untuned_score:.1%}")
    print(f"  AI-tuned score: {tuned_score:.1%}")
    delta = tuned_score - untuned_score
    if delta > 0:
        print(f"  Improvement:    +{delta:.1%}")
    elif delta == 0:
        print(f"  No change (defaults were already good for this dataset)")
    else:
        print(f"  Change:         {delta:.1%} (AI suggestion didn't help here)")
    print(f"{'=' * 60}")
else:
    print("  Could not parse AI suggestion. Try running the cell again.")

---
## What's Next? Taking RAG to Production

What we built today is a **Naive RAG** system (per Gao et al. 2024 taxonomy). Here's how real products level up:

| Level | Technique | Why |
|-------|-----------|-----|
| **Advanced RAG** | Query rewriting, reranking, HyDE | Better retrieval quality |
| **Agentic RAG** | Multi-step retrieval, tool use | Handle complex multi-hop questions |
| **Modular RAG** | Routing, adaptive retrieval | Only retrieve when needed |
| **Evaluation** | RAGAS, faithfulness metrics | Measure and improve systematically |

### Production Considerations for MBAs

- **Cost**: Embedding is free (local model). LLM costs ~$0.001-0.01 per query.
- **Latency**: Embedding ~50ms, search ~5ms, LLM ~1-3s. Total: ~2-4 seconds.
- **Scale**: sqlite-vec handles millions of vectors. For billions, consider pgvector or Pinecone.
- **Privacy**: Everything except the LLM call stays local. For full privacy, use local LLMs.

### Further Reading

- Lewis et al. (2020) — [RAG for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)
- Gao et al. (2024) — [RAG for LLMs: A Survey](https://arxiv.org/abs/2312.10997)
- Wang et al. (2024) — [Improving Text Embeddings with LLMs](https://arxiv.org/abs/2401.00368)
- Muennighoff et al. (2023) — [MTEB: Massive Text Embedding Benchmark](https://arxiv.org/abs/2210.07316)
- [sqlite-vec documentation](https://alexgarcia.xyz/sqlite-vec/)
- [fastembed documentation](https://qdrant.github.io/fastembed/)

---
*Workshop created for [MIT AI Studio](https://aiforimpact.github.io/) (MAS.664/665, EC.731, IDS.865) — Spring 2026*

*Instructor: [Brandon Sneider](https://linkedin.com/in/brandonsneider)*